In [1]:
import os

In [2]:
%pwd

'/Users/pasha/End-To-End-Chest-Cancer-Classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/pasha/End-To-End-Chest-Cancer-Classification'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Mpasha17/my-first-repo.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Mpasha17"
os.environ["MLFLOW_TRACKING_PASSWORD"]="84313Pa*"

# import dagshub
# dagshub.init(repo_owner='Mpasha17', repo_name='my-first-repo', mlflow=True)

# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/Mpasha17/my-first-repo.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
from mlflow.models.signature import infer_signature

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        model = tf.keras.models.load_model(str(path))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

            input_example = next(iter(self.valid_generator))[0]
            signature = infer_signature(input_example, self.model.predict(input_example))

            if tracking_url_type_store != "file":
                mlflow.tensorflow.log_model(
                    self.model,
                    "model",
                    registered_model_name="VGG16Model",
                    signature=signature,
                    input_example=input_example
                )
            else:
                mlflow.tensorflow.log_model(self.model, "model", signature=signature, input_example=input_example)

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e


Found 102 images belonging to 2 classes.


/opt/homebrew/Caskroom/miniconda/base/envs/cancer/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 992ms/step - accuracy: 1.0000 - loss: 0.0351
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


/opt/homebrew/Caskroom/miniconda/base/envs/cancer/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 6 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/02/13 13:37:31 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 4
Created version '4' of model 'VGG16Model'.


🏃 View run masked-mink-644 at: https://dagshub.com/Mpasha17/my-first-repo.mlflow/#/experiments/0/runs/7f0ac1e4a1ad4e1e8f45675e4675c4bc
🧪 View experiment at: https://dagshub.com/Mpasha17/my-first-repo.mlflow/#/experiments/0


Found 102 images belonging to 2 classes.


/opt/homebrew/Caskroom/miniconda/base/envs/cancer/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 1.0000 - loss: 0.0351
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


/opt/homebrew/Caskroom/miniconda/base/envs/cancer/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 6 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/02/13 13:38:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 5
Created version '5' of model 'VGG16Model'.


🏃 View run receptive-newt-555 at: https://dagshub.com/Mpasha17/my-first-repo.mlflow/#/experiments/0/runs/cdeffafc22d745a39021bd336afaf2db
🧪 View experiment at: https://dagshub.com/Mpasha17/my-first-repo.mlflow/#/experiments/0
